In [14]:
#input:


In [24]:
file = "..data/simulated_data/input_genomes/combined.fa"
total_length = 0
for line in open(file, "rb"):
    if line[0] == 62: #bit for '>'
        print(total_length)
        print(line)
        continue
    total_length += len(line.strip())
print(total_length)

NameError: name 'file' is not defined

In [34]:
import glob
import pandas as pd
import os, sys
from typing import List, Callable
def gather_reference_quantification(
        quantification_dir: str="data/simulated_data/quantification/", 
        expected_labels: List[str]=None):
    
    summary_files = glob.glob(os.path.join(quantification_dir, "*/cmseq_summation.tsv"))
    summary_labels = [x.rsplit("/",2)[1] for x in summary_files]
    
    if expected_labels:
        if not set(expected_labels) == set(summary_labels):
            err_msg = f"\
expected summary files != found summary files.\n\
Expected labels({len(expected_labels)}):{expected_labels}\n\
Found labels({len(summary_labels)}):{summary_labels}"
            raise ValueError(err_msg)
    
    dataframes = []
    for file, label in zip(summary_files, summary_labels):
        readsGB = float(label[:-2].replace("_", "."))
        df = pd.read_csv(file, sep="\t")
        df["readsGB"] = readsGB
        dataframes.append(df)        
    print(f"Gathered summary from ({len(summary_labels)}): {summary_labels}", file=sys.stderr)
    return pd.concat(dataframes)

df_ref_quant = gather_reference_quantification("../data/simulated_data/quantification/")

Gathered summary from (8): ['0_01GB', '0_02GB', '1GB', '0_002GB', '0_2GB', '0_1GB', '0_001GB', '2GB']


In [35]:
file = "../data/simulated_data/input_genomes/combined.fa"
def expected_count_eq_ratio(readsGB:List[int], input_genomes_fasta: str = "data/simulated_data/input_genomes/combined.fa"):
    total_length = 0
    for line in open(file, "rb"):
        if line[0] == 62: #bit for '>'
            #print(total_length)
            #print(line)
            continue
        total_length += len(line.strip())
    print(f"Combined genome size ({total_length})bp", file=sys.stderr)
    expected_counts = [(x*10**9)/total_length for x in readsGB]
    return expected_counts
expected_count_eq_ratio([0.1], file)

Combined genome size (16859816)bp


[5.931262832287137]

In [42]:
import numpy as np
def calculate_error_metrics(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    
    fp_inputcombined = "../data/simulated_data/input_genomes/combined.fa"
    df_out["expected_value"] = expected_count_eq_ratio(df["readsGB"], fp_inputcombined)
    df_out["expected_value_int"] = df_out["expected_value"].round(0)
    
    df_out["RAE_raw"] = np.abs(df_out["expected_value"].values-df_out["Depth median"])/df_out["expected_value"]
    df_out["RAE_int"] = np.abs(df_out["expected_value_int"].values-df_out["Depth median"])/df_out["expected_value_int"]
    
    return df_out
    #df_out["abs_err"] = 

df_summary = calculate_error_metrics(df_ref_quant)

Combined genome size (16859816)bp


In [43]:
df_summary.head(5)

,Contig,Breadth,Depth avg,Depth median,readsGB,expected_value,expected_value_int,RAE_raw,RAE_int
0,NC_014328.1.region004,0.419422,1.382508,1.0,0.01,0.593126,1.0,0.685982,0.0
1,NZ_CP053893.1.region001,0.355457,1.301931,1.0,0.01,0.593126,1.0,0.685982,0.0
2,NZ_LT906470.1.region001,0.360043,1.355237,1.0,0.01,0.593126,1.0,0.685982,0.0
3,NZ_CP053893.1.region004,0.469589,1.565359,1.0,0.01,0.593126,1.0,0.685982,0.0
4,NZ_CP053893.1.region003,0.346818,1.309063,1.0,0.01,0.593126,1.0,0.685982,0.0


In [48]:
# Visualisation
df_summary_long = df_summary.melt(
    id_vars=["Contig", "readsGB"],
    value_vars=["RAE_raw", "RAE_int"],
    var_name="Metric")
df_summary_long.head(5)

,Contig,readsGB,Metric,value
0,NC_014328.1.region004,0.01,RAE_raw,0.685982
1,NZ_CP053893.1.region001,0.01,RAE_raw,0.685982
2,NZ_LT906470.1.region001,0.01,RAE_raw,0.685982
3,NZ_CP053893.1.region004,0.01,RAE_raw,0.685982
4,NZ_CP053893.1.region003,0.01,RAE_raw,0.685982


In [59]:
import plotly.express as px
px.line(df_summary_long.sort_values("readsGB").query("Metric=='RAE_raw'"), x="readsGB", y="value", color="Contig")

In [67]:
df_summary.query("readsGB == 0.1")

,Contig,Breadth,Depth avg,Depth median,readsGB,expected_value,expected_value_int,RAE_raw,RAE_int
0,NC_014328.1.region004,0.990049,5.382567,5.0,0.1,5.931263,6.0,0.157009,0.166667
1,NZ_CP053893.1.region001,0.991744,5.140252,5.0,0.1,5.931263,6.0,0.157009,0.166667
2,NZ_LT906470.1.region001,0.987658,5.187827,5.0,0.1,5.931263,6.0,0.157009,0.166667
3,NZ_CP053893.1.region004,0.996785,5.879550,5.0,0.1,5.931263,6.0,0.157009,0.166667
4,NZ_CP053893.1.region003,0.991860,5.162646,5.0,0.1,5.931263,6.0,0.157009,0.166667
5,NZ_CP053893.1.region002,0.985503,28.251507,6.0,0.1,5.931263,6.0,0.011589,0.000000
6,NZ_CP053893.1.region005,0.992474,5.115603,5.0,0.1,5.931263,6.0,0.157009,0.166667
7,NC_014328.1.region003,0.997253,5.764677,5.0,0.1,5.931263,6.0,0.157009,0.166667
8,NZ_CP053893.1.region006,0.994557,6.305226,6.0,0.1,5.931263,6.0,0.011589,0.000000
9,NC_014328.1.region001,0.989533,4.648313,4.0,0.1,5.931263,6.0,0.325607,0.333333


In [98]:
px.line(df_summary, x="readGB",y="RAE_med", color="Contig")

In [88]:
df = df_summary.groupby("readGB").agg(
    expected = pd.NamedAgg(column="expectedCount", aggfunc="mean"),
    Depth_median = pd.NamedAgg(column="Depth median", aggfunc="mean"),
    Depth_avg = pd.NamedAgg(column="Depth avg", aggfunc="mean"),
    MSE_median = pd.NamedAgg(column="SQErr_med", aggfunc="mean"),
    MSE_avg = pd.NamedAgg(column="SQErr_avg", aggfunc="mean"),
)
df["RAE_median"] = df["MSE_avg"] / df["expected"]
df["RAE_avg"] = df["MSE_median"] / df["expected"]
df.reset_index(inplace=True)
df

,readGB,expected,Depth_median,Depth_avg,MSE_median,MSE_avg,RAE_median,RAE_avg
0,0.001,0.059313,1.0000,1.090025,0.884893,1.065897,17.970828,14.919129
1,0.002,0.118625,1.0000,1.157392,0.776821,1.125197,9.485308,6.548533
2,0.010,0.593126,1.0625,1.573637,0.278905,1.663066,2.803899,0.470229
3,0.020,1.186253,1.2500,2.130282,0.191564,2.937434,2.476230,0.161486
4,0.100,5.931263,5.1250,6.752666,0.759435,31.549487,5.319185,0.128039
5,0.200,11.862526,10.0625,13.301770,3.548686,124.758115,10.516994,0.299151
6,1.000,59.312628,50.3125,66.729243,82.217154,3146.714798,53.053032,1.386166
7,2.000,118.625257,100.9375,134.243185,317.165329,13034.348628,109.878360,2.673675


In [51]:

#add expected count:
df_summary["expectedCount"] = (df_summary["readGB"].values*10**9) / total_length

In [76]:
df_long = pd.melt(df_summary, id_vars=["Contig","readGB"], value_vars=["Breadth","Depth avg", "Depth median","expectedCount", "SQErr_avg","SQErr_med"])

In [65]:
import plotly.express as px
px.line(df_summary.sort_values("readGB"), x="readGB", y="Breadth", color="Contig")

In [67]:
df_long.head(5)

,Contig,readGB,variable,value
0,NC_014328.1.region004,0.01,Breadth,0.412592
1,NZ_CP053893.1.region001,0.01,Breadth,0.288391
2,NZ_LT906470.1.region001,0.01,Breadth,0.256180
3,NZ_CP053893.1.region004,0.01,Breadth,0.347893
4,NZ_CP053893.1.region003,0.01,Breadth,0.452192


In [78]:
fig = px.line(df_long.sort_values("readGB"),
        x="readGB",
        y="value",
        facet_row="variable",
        color="Contig"
       )
fig.update_layout(height=1800)
fig.update_yaxes(matches=None)

In [13]:
import plotly

280

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--gbk", required=True, help="input .gbk file")
parser.add_argument("--fa", help="output .file")
args = parser.parse_args(["--gbk", "out.gbk"])
args

Namespace(gbk='out.gbk', fa=None)

In [2]:
ncbi_id_file = "../config/ids_simulation_genomes.txt"

In [15]:
import os, sys

ncbi_ids = [entry.strip() for line in open(ncbi_id_file, "r").readlines() for entry in line.split()]
print(f"Found {len(ncbi_ids)} ids in file", file=sys.stderr)

expected_output = [f"../data/simulated_data/input_genomes/{x.replace('.', '_')}.fa" for x in ncbi_ids+["combined"]]

Found 5 ids in file


In [13]:
expected_output

['../data/simulated_data/input_genomes/NZ_LT906445_1.fa',
 '../data/simulated_data/input_genomes/NZ_CP053893_1.fa',
 '../data/simulated_data/input_genomes/NC_014328_1.fa',
 '../data/simulated_data/input_genomes/NZ_CP020566_1.fa',
 '../data/simulated_data/input_genomes/NZ_LT906470_1.fa',
 '../data/simulated_data/input_genomes/combined_fasta.fa']

In [14]:
[os.path.isfile(x) for x in expected_output]

[True, True, True, True, True, False]

In [3]:
str(number).replace(".","_")

'0_03'

In [16]:
gen_qsub_args(working_dir="home", runtime=1)

{'directory': 'home',
 'modules': 'tools antismash/6.1.1',
 'runtime': 1,
 'cores': 30,
 'ram': 100,
 'group': 'dtu_00009',
 'jobname': 'somefile.py',
 'output': 'home/logs/somefile_stdout',
 'error': 'home/logs/somefile_stderr'}

In [1]:
import re

In [23]:
import os
import glob
import sys


def output_exists(reads_gb: int) -> bool:
    """Checks if the output from the run already exists.
    Can be used as a check to see if the job ran succesfully,
    or if the job doesn't need to run.
    """
    reads_dir = f"../data/simulated_data/camisim/{gen_prefix(reads_gb)}/*_sample_0/reads/*"
    file_names = [x.rsplit("/",1)[1] for x in glob.glob(reads_dir)]
    n_output_fasta_files = len(re.findall("(Genome[1-5][1,2])", "".join(file_names)))
    print(f"Found ({n_output_fasta_files}/10) outputfiles", file=sys.stderr)
    return n_output_fasta_files == 10 #TODO: probably shouldn't be hardcoded.
def gen_prefix(reads_gb:int) -> str:
    return str(reads_gb).replace(".","_")+"GB"

In [24]:
output_exists(0.1)

Found (10/10) outputfiles


True

In [27]:
assemblies = f"../data/simulated_data/camisim/*GB/*_sample_0/contigs/gsa.fasta.gz"

In [29]:
camisim_dataset_GB_range = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

In [31]:
for gb in camisim_dataset_GB_range:
    antismash_html = f"../data/simulated_data/antismash/{gen_prefix(gb)}/index.html"
    print(gb, os.path.isfile(antismash_html))

1 False
0.1 False
0.01 False
0.001 False
0.0001 False
1e-05 False


In [30]:
for gb in camisim_dataset_GB_range:
    f = glob.glob(f"../data/simulated_data/camisim/{gen_prefix(gb)}/*_sample_0/contigs/gsa.fasta.gz")
    print(gb, f)

1 []
0.1 ['../data/simulated_data/camisim/0_1GB/2022.09.07_14.33.06_sample_0/contigs/gsa.fasta.gz']
0.01 []
0.001 []
0.0001 []
1e-05 []
